# AutoGen + Llama Stack Integration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meta-llama/llama-stack/blob/main/docs/notebooks/autogen/autogen_llama_stack_integration.ipynb)

## Overview

This notebook demonstrates how to use **AutoGen v0.7.5** with **Llama Stack** as the backend.

### Use Cases Covered:
1. **Two-Agent Conversation** - Teams working together on tasks
2. **Code Generation & Execution** - AutoGen generates and runs code
3. **Group Chat** - Multiple specialists collaborating  

---

## Prerequisites

```bash
# Install AutoGen v0.7.5 (new API)
pip install autogen-agentchat autogen-ext

# Llama Stack should already be running
# Default: http://localhost:8321
```

In [1]:
# Imports
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

print("✅ AutoGen imports successful")
print("Using AutoGen v0.7.5 with new team-based API")

# Check Llama Stack connectivity
import httpx

LLAMA_STACK_URL = "http://localhost:8321"

try:
    response = httpx.get(f"{LLAMA_STACK_URL}/v1/models")
    print(f"✅ Llama Stack is running at {LLAMA_STACK_URL}")
    print(f"Status: {response.status_code}")
except Exception as e:
    print(f"❌ Llama Stack not accessible: {e}")
    print("Make sure Llama Stack is running on port 8321")

✅ AutoGen imports successful
Using AutoGen v0.7.5 with new team-based API
✅ Llama Stack is running at http://localhost:8321
Status: 200


## Configuration: AutoGen v0.7.5 with Llama Stack

### How It Works

AutoGen v0.7.5 uses **OpenAIChatCompletionClient** to connect to OpenAI-compatible endpoints like Llama Stack's /v1/chat/completions.

In [3]:
# Create OpenAI-compatible client for Llama Stack
model_client = OpenAIChatCompletionClient(
    model="ollama/llama3.3:70b", # Choose any other model of your choice.
    api_key="not-needed",
    base_url="http://localhost:8321/v1",  # For pointing to llama stack end points.
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    }
)

print("✅ Model client configured for Llama Stack")
print(f"Model: ollama/llama3.3:70b")
print(f"Base URL: http://localhost:8321/v1")

✅ Model client configured for Llama Stack
Model: ollama/llama3.3:70b
Base URL: http://localhost:8321/v1


## Example 1: Simple Task with Assistant Agent

### Pattern: Single Agent Task

In v0.7.5, Autogen uses **Teams** to orchestrate agents, even for simple single-agent tasks.

**AssistantAgent:**
- AI assistant powered by Llama Stack
- Executes tasks and provides responses

### Use Case: Solve a Math Problem

In [4]:
import asyncio

# Create an AssistantAgent
assistant = AssistantAgent(
    name="MathAssistant",
    model_client=model_client,
    system_message="You are a helpful AI assistant that solves math problems. Provide clear explanations and show your work."
)

print("✅ Agent created:", assistant.name)

# Define the task
task = "What is the sum of the first 10 prime numbers? Please calculate it step by step."

# Run the task (AutoGen v0.7.5 uses async)
async def run_simple_task():
    # Create a simple team with just the assistant
    team = RoundRobinGroupChat([assistant], max_turns=1)
    result = await team.run(task=task)
    return result

# Execute in notebook
result = await run_simple_task()

print("\n" + "="*50)
print("Task Result:")
print(result.messages[-1].content if result.messages else "No response")

✅ Agent created: MathAssistant

Task Result:
To find the sum of the first 10 prime numbers, we need to follow these steps:

1. **Identify the first 10 prime numbers**: Prime numbers are natural numbers greater than 1 that have no divisors other than 1 and themselves.

2. **List the first 10 prime numbers**:
   - Start with 2 (the smallest prime number).
   - Check each subsequent natural number to see if it is divisible by any prime number less than or equal to its square root. If not, it's a prime number.
   - Continue until we have 10 prime numbers.

3. **Calculate the sum** of these numbers.

Let's list the first 10 prime numbers step by step:

1. The smallest prime number is **2**.
2. The next prime number after 2 is **3**, since it's only divisible by 1 and itself.
3. Then comes **5**, because it has no divisors other than 1 and itself.
4. Next is **7**, for the same reason as above.
5. **11** is also a prime number, as it cannot be divided evenly by any number other than 1 and it

## Example 2: Multi-Agent Team Collaboration

### Pattern: Multiple Agents Working Together

In v0.7.5, Autogen uses **RoundRobinGroupChat** to create teams where agents take turns contributing to a task.

### Use Case: Write a Technical Blog Post

In [5]:
# Create specialist agents
researcher = AssistantAgent(
    name="Researcher",
    model_client=model_client,
    system_message="You are a researcher. Provide accurate information, facts, and statistics about topics."
)

writer = AssistantAgent(
    name="Writer",
    model_client=model_client,
    system_message="You are a technical writer. Write clear, engaging content based on research provided."
)

critic = AssistantAgent(
    name="Critic",
    model_client=model_client,
    system_message="You are an editor. Review content for clarity, accuracy, and engagement. Suggest improvements."
)

print("✅ Team agents created: Researcher, Writer, Critic")

# Create a team with round-robin collaboration
async def run_blog_team():
    team = RoundRobinGroupChat([researcher, writer, critic], max_turns=12)

    task = """Write a 200-word blog post about the benefits of using Llama Stack for LLM applications.

    Steps:
    1. Researcher: Gather key information about Llama Stack
    2. Writer: Create the blog post
    3. Critic: Review and suggest improvements
    """

    result = await team.run(task=task)
    return result

# Run the team
result = await run_blog_team()

print("\n" + "="*50)
print("Final Blog Post:")
print("="*50)
# Print the last message which should contain the final output
# for msg in result.messages[-3:]:
#     print(f"\n[{msg.source}]: {msg.content[:200]}..." if len(msg.content) > 200 else f"\n[{msg.source}]: {msg.content}")
i=1
for msg in result.messages:
    print (f"Turn {i}")
    i+=1
    print(f"\n[{msg.source}]: {msg.content[:200]}..." if len(msg.content) > 200 else f"\n[{msg.source}]: {msg.content}")

✅ Team agents created: Researcher, Writer, Critic

Final Blog Post:
Turn 1

[user]: Write a 200-word blog post about the benefits of using Llama Stack for LLM applications.

    Steps:
    1. Researcher: Gather key information about Llama Stack
    2. Writer: Create the blog post
   ...
Turn 2

[Researcher]: **Unlocking Efficient LLM Applications with Llama Stack**

The Llama Stack is a cutting-edge framework designed to optimize Large Language Model (LLM) applications, offering numerous benefits for deve...
Turn 3

[Writer]: **Unlocking Efficient LLM Applications with Llama Stack**

The Llama Stack is a revolutionary framework that optimizes Large Language Model (LLM) applications, offering numerous benefits for developer...
Turn 4

[Critic]: **Reviewed Blog Post:**

The provided blog post effectively highlights the benefits of using the Llama Stack for Large Language Model (LLM) applications. However, there are a few areas that could be i...
Turn 5

[Researcher]: Here's a 200-word bl

## Example 3: Multi-Turn Task

### Pattern: Extended Team Collaboration

Use longer conversations for problem-solving where agents need multiple rounds of discussion.

### Use Case: Technical Analysis

In [6]:
# Create an analyst agent
analyst = AssistantAgent(
    name="TechAnalyst",
    model_client=model_client,
    system_message="""You are a technical analyst. Analyze technical topics deeply:
    1. Break down complex concepts
    2. Identify pros and cons
    3. Provide recommendations
    """
)

print("✅ Analyst agent created")

# Run extended analysis
async def run_analysis():
    team = RoundRobinGroupChat([analyst], max_turns=5)

    task = """Analyze the trade-offs between using local LLMs (like Llama via Llama Stack)
    versus cloud-based APIs (like OpenAI) for production applications.
    Consider: cost, latency, privacy, scalability, and maintenance."""

    result = await team.run(task=task)
    return result

result = await run_analysis()

print("\n" + "="*50)
print("Analysis Result:")
print("="*50)
i=1
for message in result.messages:
    print (f"Turn {i}")
    i+=1
    print(message.content)
    print("="*50)

✅ Analyst agent created

Analysis Result:
Turn 1
Analyze the trade-offs between using local LLMs (like Llama via Llama Stack)
    versus cloud-based APIs (like OpenAI) for production applications.
    Consider: cost, latency, privacy, scalability, and maintenance.
Turn 2
The debate between using local Large Language Models (LLMs) like Llama via Llama Stack and cloud-based APIs like OpenAI for production applications revolves around several key trade-offs. Here's a detailed analysis of the pros and cons of each approach considering cost, latency, privacy, scalability, and maintenance.

### Local LLMs (e.g., Llama via Llama Stack)

**Pros:**
1. **Privacy:** Running models locally can offer enhanced data privacy since sensitive information doesn't need to be transmitted over the internet or stored on third-party servers.
2. **Latency:** Local deployment typically results in lower latency for inference, as it eliminates the need for network requests and responses to cloud services.
3. **Cu

## Example 4: Advanced Termination Conditions

### Pattern: Code Review Loop with Stopping Logic

This example demonstrates termination using:
1. **Multiple agents** in a review loop
2. **Termination on approval** - Stops when reviewer says "LGTM"
3. **Fallback with max_turns** for safety

### Use Case: Iterative Code Review Until Approved

In [7]:
from autogen_agentchat.conditions import TextMentionTermination

# Create code review agents
code_reviewer = AssistantAgent(
    name="CodeReviewer",
    model_client=model_client,
    system_message="""You are a senior code reviewer. Review code for:
    - Bugs and edge cases
    - Performance issues
    - Security vulnerabilities
    - Best practices

    If the code looks good, say 'LGTM' (Looks Good To Me).
    If issues found, provide specific feedback for improvement."""
)

code_developer = AssistantAgent(
    name="Developer",
    model_client=model_client,
    system_message="""You are a developer. When you receive code review feedback:
    - Address ALL issues mentioned
    - Explain your changes
    - Present the improved code

    If no feedback is given, present your initial implementation."""
)

print("✅ Code review team created")

# Complex termination: Stops when reviewer approves OR max iterations reached
async def run_code_review_loop():
    # Stop when reviewer says "LGTM"
    approval_termination = TextMentionTermination("LGTM")

    team = RoundRobinGroupChat(
        [code_developer, code_reviewer],
        max_turns=16,  # Max 4 review cycles (developer + reviewer = 2 turns per cycle)
        termination_condition=approval_termination
    )

    task = """Implement a Python function to check if a string is a palindrome.

    The Developer should implement the function first.
    The Reviewer should then review it and provide feedback.
    Continue iterating until the Reviewer approves the code.
    """

    result = await team.run(task=task)
    return result

result = await run_code_review_loop()

print("\n" + "="*50)
print(f"✅ Review completed in {len(result.messages)} message(s)")
print(f"Stop reason: {result.stop_reason}")
print("="*50)

# Show the conversation flow
print("\n📝 Review Conversation Flow:")
for i, msg in enumerate(result.messages, 1):
    preview = msg.content[:150].replace('\n', ' ')
    print(f"{i}. [{msg.source}]: {preview}...")

print("\n" + "="*50)
print("Final Code (last message):")
print("="*50)
if result.messages:
    print(result.messages[-1].content)

✅ Code review team created

✅ Review completed in 5 message(s)
Stop reason: Text 'LGTM' mentioned

📝 Review Conversation Flow:
1. [user]: Implement a Python function to check if a string is a palindrome.      The Developer should implement the function first.     The Reviewer should then...
2. [Developer]: ### Initial Implementation ```python def is_palindrome(s: str) -> bool:     """     Checks if a given string is a palindrome.      Args:         s (st...
3. [CodeReviewer]: ### Code Review Feedback #### Bugs and Edge Cases  * The function does not handle non-string inputs. It should raise a `TypeError` when given a non-st...
4. [Developer]: ### Revised Implementation ```python def is_palindrome(s: str, ignore_case: bool = True, ignore_whitespace_and_punctuation: bool = True) -> bool:     ...
5. [CodeReviewer]: ### Code Review Feedback  The revised implementation has addressed all the concerns raised during the initial code review. Here's a summary of the key...

Final Code (last mess

## Example 5: Practical Team Use Case

### Pattern: Research → Write → Review Pipeline

A common pattern in content creation: research, draft, review, finalize.

### Use Case: Documentation Generator

In [8]:
# Create documentation team
doc_researcher = AssistantAgent(
    name="DocResearcher",
    model_client=model_client,
    system_message="You research technical topics and gather key information for documentation."
)

doc_writer = AssistantAgent(
    name="DocWriter",
    model_client=model_client,
    system_message="You write clear, concise technical documentation with examples."
)

print("✅ Documentation team created")

# Run documentation pipeline
async def create_documentation():
    team = RoundRobinGroupChat([doc_researcher, doc_writer], max_turns=4)
    task = """Create documentation for a hypothetical food recipe:

    Food: `Cheese Pizza`

    Include:
    - Description
    - Ingredients
    - How to make it
    - Steps
    """

    result = await team.run(task=task)
    return result

result = await create_documentation()

print("\n" + "="*50)
print("Generated Documentation:")
print("="*50)
i=1
for message in result.messages:
    print(f"Turn {i}")
    i+=1
    print(message.content)

# Turn 1: `DocResearcher` receives the task → researches the topic
# Turn 2: `DocWriter` sees the task + researcher's output → writes documentation
# Turn 3**: `DocResearcher` sees everything → can add more info
# Turn 4: `DocWriter` sees everything → refines documentation
# Stops at `max_turns=4`


✅ Documentation team created

Generated Documentation:
Turn 1
Create documentation for a hypothetical food recipe:

    Food: `Cheese Pizza`

    Include:
    - Description
    - Ingredients
    - How to make it
    - Steps
    
Turn 2
**Cheese Pizza Recipe Documentation**

### Description

A classic Cheese Pizza is a delicious and satisfying dish that consists of a crispy crust topped with a rich tomato sauce, melted mozzarella cheese, and various seasonings. This recipe provides a simple and easy-to-follow guide to making a mouth-watering Cheese Pizza at home.

### Ingredients

* **Crust:**
	+ 2 cups of warm water
	+ 1 tablespoon of sugar
	+ 2 teaspoons of active dry yeast
	+ 3 cups of all-purpose flour
	+ 1 teaspoon of salt
	+ 2 tablespoons of olive oil
* **Sauce:**
	+ 2 cups of crushed tomatoes
	+ 1/4 cup of olive oil
	+ 4 cloves of garlic, minced
	+ 1 teaspoon of dried oregano
	+ Salt and pepper to taste
* **Toppings:**
	+ 8 ounces of mozzarella cheese, shredded
	+ Fresh basil lea

### Next Steps

1. **Install autogen-ext**: `pip install autogen-agentchat autogen-ext`
2. **Start Llama Stack**: Ensure it's running on `http://localhost:8321`
3. **Experiment**: Try different team compositions and task types
4. **Explore**: Check out SelectorGroupChat and other team types

### Resources

- **AutoGen v0.7.5 Docs**: https://microsoft.github.io/autogen/
- **Llama Stack Docs**: https://llama-stack.readthedocs.io/